

# **Extraction of annotations from medical images**



**Method 1 for radiography, CT, or MRI images**

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
data_folder = '/content/drive/MyDrive/Image'
root_folder = '/content/drive/MyDrive/Image'

In [ ]:

import cv2
import numpy as np
from skimage import measure
import matplotlib.pyplot as plt
import glob

def black_contours(ct_numpy, min, max):
    inverted = max - ct_numpy
    clipped = inverted.clip(min, max)
    clipped[clipped != max] = 0
    clipped[clipped == max] = 1
    return measure.find_contours(clipped, 0.95)

#basepath = '/content/drive/MyDrive/Image'
basepath = '/content/drive/MyDrive/Image/cellul/test_cellule *.jpg'
paths = sorted(glob.glob(basepath))
print('Images found:', len(paths))

result_images = []
for c, exam_path in enumerate(paths):
    if c ==45:
        break
    im = cv2.imread(exam_path, cv2.IMREAD_GRAYSCALE) #
    blur = cv2.GaussianBlur(im, (9, 9), 0)
    _, binary_image = cv2.threshold(im, 50, 255, cv2.THRESH_BINARY)# charge l'image à partir du chemin

    gaussian_filtered = cv2.GaussianBlur(binary_image, (9, 9), 0)


    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (7, 7)) # ajustez la taille du noyau
    opened_image = cv2.morphologyEx(gaussian_filtered, cv2.MORPH_OPEN, kernel) ####
    # Appliquer une dilatation
    dilated_image = cv2.dilate(opened_image, kernel, iterations=5)
    contours = black_contours(opened_image,5, 200) # ajustez les seuils
    contour_img = np.zeros_like(im)
    result_images.append((im, contours))  # stocke l'image et les contours

for i, (im, contours) in enumerate(result_images):
    contour_img = np.zeros_like(im)

    for contour in contours:
        if len(contour) > 0:
            contour = contour[:, ::-1].astype(np.int32)

            # Filtrer les contours par taille
            if  cv2.contourArea(contour) > 650:  # ajustez la taille minimale si nécessaire
                cv2.drawContours(contour_img, [contour], -1, 255, 3)

    fig, ax = plt.subplots()
    ax.imshow(contour_img, cmap='gray')
    plt.show()



    #cv2.imwrite(f'/content/drive/MyDrive/Image/contour_image_{i+1}.png', contour_img)
    cv2.imwrite(f'/content/drive/MyDrive/Image/contour_image_{i+1}.png', contour_img)


**Technique 2 for images captured by a microscope**

In [ ]:
import cv2
import numpy as np
from skimage import measure
import matplotlib.pyplot as plt
import glob

def black_contours(ct_numpy, min, max):
    inverted = max - ct_numpy
    clipped = inverted.clip(min, max)
    clipped[clipped != max] = 0
    clipped[clipped == max] = 1
    return measure.find_contours(clipped, 0.95)

basepath = '/content/drive/MyDrive/Image/cellule/test_cellule *.jpg'
paths = sorted(glob.glob(basepath))
print('Images found:', len(paths))

result_images = []
for c, exam_path in enumerate(paths):
    if c ==45:
        break
    im = cv2.imread(exam_path, cv2.IMREAD_GRAYSCALE)
    blur = cv2.GaussianBlur(im, (9, 9), 0)
    _, binary_image = cv2.threshold(im, 50, 255, cv2.THRESH_BINARY)
    gaussian_filtered = cv2.GaussianBlur(binary_image, (5, 5), 0)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (7, 7))
    opened_image = cv2.morphologyEx(gaussian_filtered, cv2.MORPH_OPEN, kernel)
    dilated_image = cv2.dilate(gaussian_filtered, kernel, iterations=1)
    contours = black_contours(gaussian_filtered,150, 200)
    contour_img = np.zeros_like(im)
    result_images.append((im, blur, binary_image, gaussian_filtered, opened_image, dilated_image, contours))

for i, (im, blur, binary_image, gaussian_filtered, opened_image, dilated_image, contours) in enumerate(result_images):
    contour_img = np.zeros_like(im)

    for contour in contours:
        if len(contour) > 0:
            contour = contour[:, ::-1].astype(np.int32)

            # Filtrer les contours par taille
            if cv2.contourArea(contour) > 850:
                cv2.drawContours(contour_img, [contour], -1, 255, 3)

    fig, axs = plt.subplots(2, 4, figsize=(60, 60))
    axs[0,0].imshow(im, cmap='gray')
    axs[0,0].set_title('Original Image')
    axs[0,1].imshow(blur, cmap='gray')
    axs[0,1].set_title('Image floue')
    axs[0,2].imshow(binary_image, cmap='gray')
    axs[0,2].set_title('Image binaire')
    axs[0,3].imshow(gaussian_filtered, cmap='gray')
    axs[0,3].set_title('Image filtrée gaussienne')
    axs[1,0].imshow(opened_image, cmap='gray')
    axs[1,0].set_title('Image ouverte')
    axs[1,1].imshow(dilated_image, cmap='gray')
    axs[1,1].set_title('Dilated Image')
    axs[1,2].imshow(contour_img, cmap='gray')
    axs[1,2].set_title('Image dilatée')
    axs[1,3].axis('off')  # Hide last subplot

    filename = f'/content/drive/MyDrive/Image/cellule/test222/steps_image_{i+1}.png'
    plt.savefig(filename)
    plt.show()
    print(f"Figure saved as {filename}")






